In [1]:
import math

In [2]:
def cor(alpha,beta,mu,eta,L,z):
    a1_sq = 1 + math.tan(mu)**2 * math.cos(alpha)**2
    a2_sq = 1 + math.tan(eta)**2 * math.cos(beta)**2

    a1 = math.sqrt(a1_sq)
    a2 = math.sqrt(a2_sq)

    num_l2 = (L * math.cos(alpha) / a1) -z
    den_l2 = (math.cos(alpha) / a1) + (math.cos(beta) / a2)
    l2 = num_l2 / den_l2
    l1 = L - l2


    # Equation for l1x^2
    l1x_sq = (l1**2) / a1_sq

    # Equation for l1y^2
    l1y_sq = (l1**2) / (math.sin(mu)**2 + (math.cos(mu) / math.cos(alpha))**2)

    # Equation for l2x^2
    l2x_sq = (l2**2) / a2_sq

    # Equation for l2y^2
    l2y_sq = (l2**2) / (math.sin(eta)**2 + (math.cos(eta) / math.cos(beta))**2)

    return {
        "l1": l1, "l2": l2,
        "l1x_sq": l1x_sq, "l1y_sq": l1y_sq,
        "l2x_sq": l2x_sq, "l2y_sq": l2y_sq
    }


In [4]:
import math

def run_tests():
    print("Running tests for cor() function...\n")

    # --- Test 1: Zero angles ---
    # Cable straight down, L = 100, ROV depth z = 40
    res1 = cor(0, 0, 0, 0, 100, 40)
    print(f"Test 1 (Zero angles): l1={res1['l1']:.2f}, l2={res1['l2']:.2f}")
    assert math.isclose(res1['l2'], 30.0), f"Expected l2=30, got {res1['l2']}"
    assert math.isclose(res1['l1'], 70.0), f"Expected l1=70, got {res1['l1']}"

    # --- Test 2: Small angles ---
    # Slight tilt
    res2 = cor(math.radians(10), math.radians(5), math.radians(8), math.radians(3), 100, 50)
    print(f"Test 2 (Small angles): l1={res2['l1']:.2f}, l2={res2['l2']:.2f}")
    assert res2['l1'] > 0 and res2['l2'] > 0, "Lengths should be positive"

    # --- Test 3: Symmetric split ---
    # Want l1 = l2 = 50 for L = 100
    alpha = beta = math.radians(0.3)
    mu = eta = math.radians(0.1)
    a1 = math.sqrt(1 + math.tan(mu)**2 * math.cos(alpha)**2)
    a2 = math.sqrt(1 + math.tan(eta)**2 * math.cos(beta)**2)
    l2_target = 50
    z_target = 100 * math.cos(alpha)/a1 - l2_target * (math.cos(alpha)/a1 + math.cos(beta)/a2)

    res3 = cor(alpha, beta, mu, eta, 100, z_target)
    print(f"Test 3 (Symmetric split): l1={res3['l1']:.2f}, l2={res3['l2']:.2f}")
    assert math.isclose(res3['l1'], 50.0, rel_tol=1e-5), f"Expected l1=50, got {res3['l1']}"
    assert math.isclose(res3['l2'], 50.0, rel_tol=1e-5), f"Expected l2=50, got {res3['l2']}"

    # --- Test 4: Random angles ---
    res4 = cor(math.radians(25), math.radians(15), math.radians(10), math.radians(5), 120, 60)
    print(f"Test 4 (Random angles): l1={res4['l1']:.2f}, l2={res4['l2']:.2f}")
    assert res4['l1'] > 0 and res4['l2'] > 0, "Lengths must be positive"
    assert res4['l1x_sq'] > 0 and res4['l2y_sq'] > 0, "Projection squares must be positive"

    print("\nAll tests passed!")

# Run the tests
run_tests()


Running tests for cor() function...

Test 1 (Zero angles): l1=70.00, l2=30.00
Test 2 (Small angles): l1=75.87, l2=24.13
Test 3 (Symmetric split): l1=50.00, l2=50.00
Test 4 (Random angles): l1=94.48, l2=25.52

All tests passed!


In [3]:
def calculate_coordinates(alpha, beta, mu, eta, L, z_input, sb=1, l0=0):
    results = cor(alpha, beta, mu, eta, L, z_input, sb, l0)

    #Extract linear components (square roots)
    l1x = math.sqrt(results["l1x_sq"])
    l1y = math.sqrt(results["l1y_sq"])
    l2x = math.sqrt(results["l2x_sq"])
    l2y = math.sqrt(results["l2y_sq"])

    #Calculate x and y (Equation 3)
    x = l1x * math.sin(alpha) + l2x * math.sin(beta)
    y = l1y * math.sin(mu) + l2y * math.sin(eta)

    #Calculate z
    z_alpha_beta = l0 + sb * l1x * math.cos(alpha) - sb * l2x * math.cos(beta)
    z_mu_eta = l0 + sb * l1y * math.cos(mu) - sb * l2y * math.cos(eta)

    return {
        "x": x,
        "y": y,
        "z_check_1": z_alpha_beta,
        "z_check_2": z_mu_eta,
        "components": {"l1x": l1x, "l1y": l1y, "l2x": l2x, "l2y": l2y}
    }